# cnn 神经网络实现
不使用pytorch 实现cnn
需要在 backward 的基础之上 加上 卷积层、池化层的实现就好

## 卷积层
卷积层有两个东西，一个是data 一个是filter(kernel),整个流程如下图所示
<img src="卷积运算.jpg" height=960>

### 填充padding
使用填充主要是为了调整输出的大小。比如,对大小为 (4, 4) 的输入  数据应用 (3, 3) 的滤波器时,输出大小变为 (2, 2),相当于输出大小  比输入大小缩小了 2 个元素。这在反复进行多次卷积运算的深度网  络中会成为问题。为什么呢?因为如果每次进行卷积运算都会缩小  空间,那么在某个时刻输出大小就有可能变为 1,导致无法再应用  卷积运算。为了避免出现这样的情况,就要使用填充。在刚才的例  子中,将填充的幅度设为 1,那么相对于输入大小 (4, 4),输出大小  也保持为原来的 (4, 4)。因此,卷积运算就可以在保持空间大小不变  的情况下将数据传给下一层。

### channel
当图片不是灰色，而是rgb 3通道的数据时，filter 也应该是3通道的，这样卷积出来的会坍缩成二维数据,如图所示。
<br>
<img src="n_channel.jpg">
<br>
如果你想卷积后的通道数和卷积前的一样，那么应该使用多个filter，并且每个filter都是和图片相同的通道数，如图所示
<br>
<img src="n_filter.jpg">

## 池化层
池化是缩小高、长方向上的空间的运算
<br>
<img src="max_pool.jpg">


## im2col and col2im
卷积层和池化层的前向传播和后向传播需要用到的两个方法，用于简化计算
原理：im2col 是把 3维图像按照每个卷积核的大小，每一个都展开成一个col,再把每个3通道的filter 展开成一列。然后卷积就成为了矩阵乘法
一个3通道图像会变成一个二维矩阵，每一行都是需要进行卷积运算的值，然后3通道的filter也变成了一个二维矩阵，每一列都是一个filter。这两个矩阵进行矩阵乘法，然后需要把得到的新矩阵做reshape->新矩阵的每一列都是一次卷积的结果，都应该reshape成一个OHxOW的矩阵，然后如果是有fn个filter的，新的矩阵应该会有fn列，就要reshape成fnxOHxOW
<br>
<img src="img/im2col.jpg">
<br>

<img src="img/im2col_all.jpg">

In [182]:
import pickle
import numpy as np
from collections import OrderedDict

import os, time, random
from typing import List, Tuple
from PIL import Image, ImageOps,ImageEnhance, ImageChops, ImageFilter
from tqdm import tqdm
import json

In [183]:
def softmax(z: np.ndarray) -> np.ndarray:
    z = z - z.max(axis=1, keepdims=True)
    ez = np.exp(z, dtype=np.float32)
    return ez / (np.sum(ez, axis=1, keepdims=True) + 1e-12)

def cross_entropy_loss(probs: np.ndarray, y_true_idx: np.ndarray):
    """
    probs: (N, C) after softmax
    y_true_idx: (N,) integer labels
    returns (loss, grad_logits)
    """
    N = probs.shape[0]
    logp = -np.log(probs[np.arange(N), y_true_idx] + 1e-12)
    loss = float(np.mean(logp))
    return loss

In [184]:
class Adam:

    """Adam (http://arxiv.org/abs/1412.6980v8)"""

    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None
        
    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
        
        self.iter += 1
        lr_t  = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)         
        
        for key in params.keys():
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])
            self.v[key] += (1 - self.beta2) * (grads[key]**2 - self.v[key])
            
            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)

In [185]:
def augment_pil(img, img_size):
    # 随机水平/垂直翻转
    if random.random() < 0.5:
        img = img.transpose(Image.FLIP_LEFT_RIGHT)
    if random.random() < 0.3:
        img = img.transpose(Image.FLIP_TOP_BOTTOM)

    # 随机旋转 ±15°
    if random.random() < 0.7:
        angle = random.uniform(-15, 15)
        img = img.rotate(angle, resample=Image.BILINEAR, fillcolor=0)

    # 随机平移（模拟位置偏移）
    if random.random() < 0.6:
        w, h = img.size
        max_dx = int(0.1 * w)
        max_dy = int(0.1 * h)
        dx = random.randint(-max_dx, max_dx)
        dy = random.randint(-max_dy, max_dy)
        img = ImageChops.offset(img, dx, dy)
        img = img.crop((max_dx, max_dy, w-max_dx, h-max_dy))  # crop back
        img = img.resize((img_size, img_size), Image.BILINEAR)

    # 随机亮度/对比度
    if random.random() < 0.6:
        factor = random.uniform(0.7, 1.4)
        img = ImageEnhance.Brightness(img).enhance(factor)
    if random.random() < 0.6:
        factor = random.uniform(0.7, 1.4)
        img = ImageEnhance.Contrast(img).enhance(factor)

    # 轻微高斯模糊（防锐化过拟合）
    if random.random() < 0.2:
        img = img.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.3, 0.8)))

    # 随机擦除（CoarseDropout 模拟）
    if random.random() < 0.3:
        arr = np.array(img)
        h, w = arr.shape
        for _ in range(random.randint(1, 6)):
            x = random.randint(0, w-4)
            y = random.randint(0, h-4)
            arr[y:y+random.randint(1,4), x:x+random.randint(1,4)] = 0
        img = Image.fromarray(arr)

    return img

In [186]:

def im2col(input_data, kernel_h, kernel_w, stride=1, pad=0):
    """
    将输入转换为列矩阵，每个 filter 大小的块展平为一行。
    
    input_data: shape (N, C, H, W)
    返回: shape (N * out_h * out_w, C * kernel_h * kernel_w)
    """
    N, C, H, W = input_data.shape

    out_h = (H + 2 * pad - kernel_h) // stride + 1
    out_w = (W + 2 * pad - kernel_w) // stride + 1

    # 填充
    img = np.pad(input_data, ((0, 0), (0, 0), (pad, pad), (pad, pad)), mode='constant')

    # 初始化 col 矩阵
    col = np.zeros((N, C, kernel_h, kernel_w, out_h, out_w))

    # 利用步进提取每个窗口
    # 这一步很有趣，虽然说是每一次把filter对应的img data 铺平成一行，但是这边不是这样做的
    for i in range(kernel_h):
        i_max = i + stride * out_h
        for j in range(kernel_w):
            j_max = j + stride * out_w
            col[:, :, i, j, :, :] = img[:, :, i:i_max:stride, j:j_max:stride]
        

    # 转置 + reshape：把 (N, out_h, out_w, C, K_h, K_w) -> (N*out_h*out_w, C*K_h*K_w)
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N * out_h * out_w, -1)
    return col

def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """

    Parameters
    ----------
    col :
    input_shape : 输入数据的形状（例：(10, 1, 28, 28)）
    filter_h :
    filter_w
    stride
    pad

    Returns
    -------

    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

In [187]:
class Convolution:
    def __init__(self, K, b, stride = 1, pad = 0):
        self.K = K
        self.b = b
        self.stride = stride
        self.pad = pad
        
        # 中间数据（backward时使用）
        self.x = None   
        self.col = None
        self.col_K = None
        
        # 权重和偏置参数的梯度
        self.dK = None
        self.db = None
    
    def forward(self, x):
        fn, c, kh, kw = self.K.shape
        N,C,H,W = x.shape
        out_h = (H + 2* self.pad -kh)//self.stride + 1
        out_w = (W + 2* self.pad -kw)//self.stride + 1
        col = im2col(x,kh, kw, self.stride, self.pad)
        col_K = self.K.reshape(fn, -1).T
        out = col@col_K + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        self.x = x
        self.col = col
        self.col_K = col_K
        
        return out
    
    def backward(self,dout):
        FN, C, FH, FW = self.K.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dK = np.dot(self.col.T, dout)
        self.dK = self.dK.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = dout @ self.col_K.T
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx
        

In [188]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max

        return out

    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx

In [189]:
class simpleConvNet:
    # conv -> relu -> pool ->conv->relu->pool->full->relu->full
        def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_nums':[8, 16], 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=64, output_size=5, weight_init_std=0.01):
            filter_nums = conv_param['filter_nums']
            filter_size = conv_param['filter_size']
            filter_pad = conv_param['pad']
            filter_stride = conv_param['stride']
            input_size = input_dim[1]
    
            # 初始化权重
            self.params = {}
            self.params['K1'] = np.random.randn(filter_nums[0], input_dim[0], filter_size, filter_size).astype(np.float32) * np.sqrt(2.0 / (filter_size * filter_size * input_dim[0]))
            
            self.params['b1'] = np.zeros(filter_nums[0])
            self.params['K2'] = np.random.randn(filter_nums[1], filter_nums[0], filter_size, filter_size).astype(np.float32) * np.sqrt(2.0 / (filter_size * filter_size * filter_nums[0]))
            self.params['b2'] = np.zeros(filter_nums[1])
            
            s1 = input_size - filter_size + 1
            s1p = s1 // 2
            s2 = s1p - filter_size + 1
            s2p = s2 // 2
            self.flat_dim = s2p * s2p * filter_nums[1]
            
            self.params['W1'] = np.random.randn(self.flat_dim, hidden_size).astype(np.float32) * np.sqrt(2.0 / self.flat_dim)
            self.params['b3'] = np.zeros((hidden_size,), dtype=np.float32)
            self.params['W2'] = np.random.randn(hidden_size, output_size).astype(np.float32) * np.sqrt(2.0 / hidden_size)
            self.params['b4'] = np.zeros((output_size,), dtype=np.float32)
    
            # 生成层
            self.layers = OrderedDict()
            self.layers['Conv1'] = Convolution(self.params['K1'], self.params['b1'],
                                               filter_stride, filter_pad)
            self.layers['Relu1'] = Relu()
            self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
            self.layers['Conv2'] = Convolution(self.params['K2'], self.params['b2'], filter_stride, filter_pad)
            self.layers['Relu2'] = Relu()
            self.layers['Pool2'] = Pooling(pool_h=2, pool_w=2, stride=2)
            self.layers['Affine1'] = Affine(self.params['W1'], self.params['b3'])
            self.layers['Relu3'] = Relu()
            self.layers['Affine2'] = Affine(self.params['W2'], self.params['b4'])
    
            self.last_layer = SoftmaxWithLoss()
    
        def predict(self, x):
            for layer in self.layers.values():
                x = layer.forward(x)
    
            return x
    
        def loss(self, x, t):
            """求损失函数
            """
            y = self.predict(x)
            return y, self.last_layer.forward(y, t)
    
        def accuracy(self, x, t, batch_size=100):
            if t.ndim != 1 : t = np.argmax(t, axis=1)
            
            acc = 0.0
            
            for i in range(int(x.shape[0] / batch_size)):
                tx = x[i*batch_size:(i+1)*batch_size]
                tt = t[i*batch_size:(i+1)*batch_size]
                y = self.predict(tx)
                y = np.argmax(y, axis=1)
                acc += np.sum(y == tt) 
            
            return acc / x.shape[0]
    
        def gradient(self, x, t):
            """求梯度（误差反向传播法）
    
            Parameters
            ----------
            x : 输入数据
            t : 教师标签
    
            Returns
            -------
            具有各层的梯度的字典变量
                grads['W1']、grads['W2']、...是各层的权重
                grads['b1']、grads['b2']、...是各层的偏置
            """
    
            # backward
            dout = 1
            dout = self.last_layer.backward(dout)
    
            layers = list(self.layers.values())
            layers.reverse()
            for layer in layers:
                dout = layer.backward(dout)
    
            # 设定
            grads = {}
            grads['K1'], grads['b1'] = self.layers['Conv1'].dK, self.layers['Conv1'].db
            grads['K2'], grads['b2'] = self.layers['Conv2'].dK, self.layers['Conv2'].db
            grads['W1'], grads['b3'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
            grads['W2'], grads['b4'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
    
            return grads
            
        def save_params(self, file_name="params.pkl"):
            params = {}
            for key, val in self.params.items():
                params[key] = val
            with open(file_name, 'wb') as f:
                pickle.dump(params, f)

In [190]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

class Affine:
    def __init__(self, W, b):
        self.W =W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 权重和偏置参数的导数
        self.dW = None
        self.db = None

    def forward(self, x):
        # 对应张量
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 还原输入数据的形状（对应张量）
        return dx
    
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None # softmax的输出
        self.t = None # 监督数据

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss= cross_entropy_loss(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 监督数据是one-hot-vector的情况
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

In [191]:
# 一些工具
# === Simple dataset loader for folder structure ===
IMG_SIZE = 100

def accuracy(logits, y):
    preds = np.argmax(softmax(logits), axis=1)
    return float(np.mean(preds == y))

def list_images(root: str) -> Tuple[list, list]:
    classes = sorted([d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))])
    items = []
    for ci, cname in enumerate(classes):
        cdir = os.path.join(root, cname)
        for fn in os.listdir(cdir):
            if fn.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                items.append((os.path.join(cdir, fn), ci))
    return items, classes

def load_batch(paths_labels, start, end, img_size=IMG_SIZE, augment=False):
    batch = paths_labels[start:end]
    N = len(batch)
    X = np.zeros((N, 1,img_size, img_size), dtype=np.float32)
    y = np.zeros((N,), dtype=np.int64)
    for i, (p, lab) in enumerate(batch):
        img = Image.open(p).convert('L')  # grayscale
        if augment:
           # img = augment_pil(img, img_size)
            # 1. 水平翻转
            if random.random() < 0.5:
                img = img.transpose(Image.FLIP_LEFT_RIGHT)

            # 2. 旋转 (-15° ~ +15°)
            if random.random() < 0.4:
                angle = random.uniform(-15, 15)
                img = img.rotate(angle, resample=Image.BILINEAR, fillcolor=0)

            # 3. 缩放 (0.9x ~ 1.1x)
            if random.random() < 0.4:
                scale = random.uniform(0.9, 1.1)
                w, h = img.size
                new_w, new_h = int(w * scale), int(h * scale)
                img = img.resize((new_w, new_h), Image.BILINEAR)
                # 裁剪或填充回原尺寸（这里用 center crop + pad）
                if scale > 1.0:
                    # zoom in: center crop
                    left = (new_w - w) // 2
                    top = (new_h - h) // 2
                    img = img.crop((left, top, left + w, top + h))
                else:
                    # zoom out: pad with black
                    img = ImageOps.pad(img, (w, h), color=0, centering=(0.5, 0.5))

            # 4. 平移 (-10% ~ +10% of size)
            if random.random() < 0.4:
                w, h = img.size
                tx = random.uniform(-0.1, 0.1) * w
                ty = random.uniform(-0.1, 0.1) * h
                img = img.transform(
                    (w, h),
                    Image.AFFINE,
                    (1, 0, tx, 0, 1, ty),
                    resample=Image.BILINEAR,
                    fillcolor=0
                )

            # 5. 剪切 (shear, -10° ~ +10°)
            if random.random() < 0.3:
                shear_angle = random.uniform(-10, 10)  # degrees
                shear_factor = np.tan(np.radians(shear_angle))
                w, h = img.size
                img = img.transform(
                    (w, h),
                    Image.AFFINE,
                    (1, shear_factor, 0, 0, 1, 0),
                    resample=Image.BILINEAR,
                    fillcolor=0
                )
                
        img = img.resize((img_size, img_size), Image.BILINEAR)
        X[i] = np.expand_dims(np.asarray(img, dtype=np.float32) / 255.0, axis=0)
        y[i] = lab
    return X, y

def state_dict(model):
    """
    Collect all top-level numpy array attributes from the model.
    Returns a dict: {name: np.ndarray}
    """
    sd = {}
    for name, val in vars(model).items():
        if isinstance(val, np.ndarray):
            sd[name] = val
    return sd

def load_state_dict(model, state, strict=True):
    """
    Load arrays from `state` (a dict of name->ndarray) into model attributes.
    If strict=True, raises on missing keys or shape mismatch.
    Returns a report dict.
    """
    report = {"loaded": [], "missing_in_model": [], "missing_in_state": [], "shape_mismatch": []}
    model_vars = vars(model)
    # apply
    for k, arr in state.items():
        if k not in model_vars:
            report["missing_in_model"].append(k)
            if strict:
                raise KeyError(f"Parameter '{k}' not found in model.")
            continue
        if not isinstance(model_vars[k], np.ndarray):
            report["shape_mismatch"].append((k, "not an ndarray on model"))
            if strict:
                raise TypeError(f"Model attribute '{k}' is not an ndarray.")
            continue
        if model_vars[k].shape != arr.shape:
            report["shape_mismatch"].append((k, (model_vars[k].shape, arr.shape)))
            if strict:
                raise ValueError(f"Shape mismatch for '{k}': model {model_vars[k].shape} vs state {arr.shape}")
            # non-strict: still load
        setattr(model, k, arr.astype(model_vars[k].dtype, copy=True))
        report["loaded"].append(k)
    # find params that were expected but not provided
    for k, v in model_vars.items():
        if isinstance(v, np.ndarray) and k not in state:
            report["missing_in_state"].append(k)
    return report

In [192]:

def train():
    data_root = 'dataset'
    epochs = 30
    batch_size = 16
    img_size = 100
    use_augment = True
    lr=0.03
    
    t0_total = time.time()
    
    train_items, classes = list_images(os.path.join(data_root, 'train'))
    val_items, _ = list_images(os.path.join(data_root, 'val'))
    num_classes = len(classes)
    print(f'Classes: {classes} (num_classes={num_classes})')
    
    network = simpleConvNet(input_dim=(1,100,100), 
                        conv_param = {'filter_nums': [8,16], 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=64, output_size=5, weight_init_std=0.01)
    
    last_train_loss, last_val_loss = None, None
    
    # ---- Prepare save path ----

    for ep in range(1, epochs+1):
        print(f"\n=== Epoch {ep}/{epochs} ===")
        random.shuffle(train_items)

        # ---- Train ----
        t0 = time.time()
        loss_accum, acc_accum, seen = 0.0, 0.0, 0
        pbar = tqdm(range(0, len(train_items), batch_size), desc=f"Train {ep}", ncols=80)
        for s in pbar:
            X, y = load_batch(train_items, s, min(s+batch_size, len(train_items)), img_size, augment=use_augment)
            logits,loss = network.loss(X, y)
            
            grads = network.gradient(X, y)
            # adam = Adam(lr=lr)
            # adam.update(network.params, grads = grads)
            for key in network.params.keys():
                network.params[key] -= lr * grads[key] 
                
            
            #loss, dlogits = cross_entropy_loss(probs, y)
            #model.backward(dlogits, cache)

            acc = accuracy(logits, y)
            bsz = X.shape[0]
            loss_accum += loss * bsz
            acc_accum += acc * bsz
            seen += bsz
            pbar.set_postfix(loss=f"{loss:.3f}", acc=f"{acc*100:.1f}%")

        last_train_loss = loss_accum / max(1, seen)
        train_time = time.time() - t0
        print(f"Epoch {ep:02d} Train: loss={last_train_loss:.4f}, acc={acc_accum/seen:.3f}, time={train_time:.1f}s")

        # ---- Val ----
        t1 = time.time()
        val_loss, val_acc, vseen = 0.0, 0.0, 0
        pbar_val = tqdm(range(0, len(val_items), batch_size), desc=f"Val {ep}", ncols=80)
        for s in pbar_val:
            Xv, yv = load_batch(val_items, s, min(s+batch_size, len(val_items)), img_size, augment=False)
            # logits, _ = model.forward(Xv)
            # probs = softmax(logits)
            # l, _ = cross_entropy_loss(probs, yv)
            
            logits, l = network.loss(Xv, yv)
            a = accuracy(logits, yv)
            bsz = Xv.shape[0]
            val_loss += l * bsz
            val_acc += a * bsz
            vseen += bsz
            pbar_val.set_postfix(loss=f"{l:.3f}", acc=f"{a*100:.1f}%")

        last_val_loss = val_loss / max(1, vseen)
        val_time = time.time() - t1
        print(f"Epoch {ep:02d} Val  : loss={last_val_loss:.4f}, acc={val_acc/vseen:.3f}, time={val_time:.1f}s")

    total_time = time.time() - t0_total
    print(f"\n✅ Training finished in {total_time/60:.2f} minutes.")
    print(f"🔚 Final losses — train: {last_train_loss:.4f}, val: {last_val_loss:.4f}")
    

train()

Classes: ['bird', 'car', 'cat', 'dog', 'person'] (num_classes=5)

=== Epoch 1/30 ===


Train 1: 100%|███████████| 32/32 [00:04<00:00,  6.75it/s, acc=25.0%, loss=1.679]


Epoch 01 Train: loss=1.7219, acc=0.258, time=4.7s


Val 1: 100%|██████████████| 16/16 [00:00<00:00, 17.57it/s, acc=0.0%, loss=1.844]


Epoch 01 Val  : loss=1.6105, acc=0.272, time=0.9s

=== Epoch 2/30 ===


...

Epoch 29 Val  : loss=1.5796, acc=0.380, time=1.0s

=== Epoch 30/30 ===


Train 30: 100%|██████████| 32/32 [00:04<00:00,  6.61it/s, acc=75.0%, loss=0.934]


Epoch 30 Train: loss=1.1930, acc=0.514, time=4.8s


Val 30: 100%|████████████| 16/16 [00:01<00:00, 15.69it/s, acc=30.0%, loss=1.504]

Epoch 30 Val  : loss=1.5757, acc=0.376, time=1.0s

✅ Training finished in 2.88 minutes.
🔚 Final losses — train: 1.1930, val: 1.5757
